In [584]:
import numpy as np
import pandas as pd
import itertools
from shapely.geometry import Polygon
import math

In [585]:
# to_gk git
# to_1992 git
# to_2000 git
# pola git
# 
# gk_to_geo, gk_to_geo2 git
# get_mgk, get_mgk2 git
# 
# skale, pola git

## Funkcje

In [586]:
def to_gk(fi, lam, lam_zero, a=6378137, e2=0.00669437999013):
    """przelicza współrzędne fi, lambda danej elipsoidy na x,y Gaussa-Krugera (w metrach), lam_zero to południk osiowy.
    fi, lam, lam_zero podajemy w stopniach."""
    a2 = a*a
    b2 = a2*(1 - e2)
    eprim2 = (a2 - b2) / b2 
    fi = np.deg2rad(fi)
    lam = np.deg2rad(lam)
    lam_zero = np.deg2rad(lam_zero)
    t = np.tan(fi)
    eta2 = eprim2 * (np.cos(fi))**2
    l = lam - lam_zero
    N = a / np.sqrt(1 - e2 * np.sin(fi) ** 2)
    A0 = 1 - (e2/4) - (3*(e2**2)/64) - (5*(e2**3)/256)
    A2 = (3/8) * (e2 + ((e2**2)/4) + (15*(e2**3))/128)
    A4 = (15/256) * (e2**2 + 3*(e2**3)/4)
    A6 = 35*(e2**3)/3072
    sigma = a * ( A0*fi - A2*np.sin(2*fi) + A4*np.sin(4*fi) - A6*np.sin(6*fi) )
    xgk = sigma + ((l**2)/2)*N*np.sin(fi)*np.cos(fi)* ( 1 + ((l**2)/12)*(np.cos(fi)**2)*(5-t**2+9*eta2+4*(eta2**2)) + ((l**4)/360)*(np.cos(fi)**4)*(61-58*t**2+t**4+270*eta2-330*eta2*t**2) )
    ygk = l*N*np.cos(fi)* ( 1 + ((l**2)/6)*(np.cos(fi)**2)*(1-t**2+eta2) + ((l**4)/120)*(np.cos(fi)**4)*(5-18*t**2+t**4+14*eta2-58*eta2*t**2) )
    return xgk, ygk

In [587]:
def to_1992(fi, lam):
    """przelicza fi, lam w stopniach na układ 1992 (x,y w metrach). Końcowe współrzędne mają 6 cyfr w 1992."""
    # m0_92 - skala długości na południku osiowym 19 stopni
    m0_92 = 0.9993
    xgk, ygk = to_gk(fi, lam, 19)
    # cechowanie współrzędnych
    x92 = m0_92*xgk - 5_300_000
    y92 = m0_92*ygk + 500_000
    return x92, y92

In [588]:
def to_2000(fi, lam):
    """przelicza fi, lam w stopniach na układ 2000 (x,y w metrach). Końcowe współrzędne mają 7 cyfr w 2000.
    Są cztery strefy odwzorowawcze. południki osiowe 15, 18, 21, 24. Pasy oznaczone numerami: 5, 6, 7, 8.
    m0 południka osiowego każdego pasa wynosi 0.999923."""
    if abs(lam-15) <= 1.5:
        osiowy = 15
    elif abs(lam-18) <= 1.5:
        osiowy = 18
    elif abs(lam-21) <= 1.5:
        osiowy = 21
    elif abs(lam-24) <= 1.5:
        osiowy = 24
        
    m0_2000 = 0.999923
    nr_strefy = osiowy/3
    xgk, ygk = to_gk(fi, lam, osiowy)
    x2000 = m0_2000 * xgk
    y2000 = m0_2000 * ygk + nr_strefy*1_000_000 + 500_000
    # f"{round(x2000, 3):.3f}"
    return x2000, y2000

In [589]:
def pole_czworokata(fi1, lam1, fi2, lam2, e2=0.00669437999013, a=6378137):  # , b=6356752.314140347
    """podajemy dwa narożniki siatki, dostajemy pole w metrach. pole na elipsoidzie."""
    b = a * (1 - e2)**0.5
    e = np.sqrt(e2)
    lam1 = np.deg2rad(lam1)
    lam2 = np.deg2rad(lam2)
    fi1 = np.deg2rad(fi1)
    fi2 = np.deg2rad(fi2)
    return abs(b**2*(lam2 - lam1)/2 *  (  ( np.sin(fi2)/(1 - e2*np.sin(fi2)**2) + (1/(2*e))*np.log((1 + e*np.sin(fi2)) / (1 - e*np.sin(fi2))) )  -  ( np.sin(fi1)/(1 - e2*np.sin(fi1)**2) + (1/(2*e))*np.log((1 + e*np.sin(fi1)) / (1 - e*np.sin(fi1))) )  ))

In [590]:
def pole_poligonu(lista_punktow):
    """Podajemy punkty kolejno. 'Rysujemy granice poligonu'. Działamy na płaszczyźnie."""
    return Polygon(lista_punktow).area

In [591]:
def gk_to_geo(xgk, ygk, lam_zero, a=6378137, e2=0.00669437999013):
    """przliecza wsp w układzie gaussa krugera(metry) na geodezyjne(stopnie). lam_zero to południk osiowy(stopnie).
    Wzory z pdfa jakiegos"""
    A0 = 1 - (e2/4) - (3*(e2**2)/64) - (5*(e2**3)/256)
    A2 = (3/8) * (e2 + ((e2**2)/4) + (15*(e2**3))/128)
    A4 = (15/256) * (e2**2 + 3*(e2**3)/4)
    A6 = 35*(e2**3)/3072
    epsilon = 0.00001/3600  # epsilon w stopniach dziesiętnych
    epsilon = np.deg2rad(epsilon) # epsilon w radianach

    fi = xgk/(a*A0)
    sigma = a * ( A0*fi - A2*np.sin(2*fi) + A4*np.sin(4*fi) - A6*np.sin(6*fi) )
    while True:
        fi_stare = fi
        fi = fi + (xgk-sigma)/(a*A0)
        if abs(fi-fi_stare) < epsilon/3600:
            break
        sigma = a * ( A0*fi - A2*np.sin(2*fi) + A4*np.sin(4*fi) - A6*np.sin(6*fi) )

    t = np.tan(fi)
    a2 = a*a
    b2 = a2*(1 - e2)
    eprim2 = (a2 - b2) / b2
    eta2 = eprim2 * (np.cos(fi))**2
    N = a / np.sqrt(1 - e2 * np.sin(fi) ** 2)
    M = (a*(1-e2)) / ( (1 - e2 * np.sin(fi) ** 2) ** 1.5 )
    
    fi_g = fi - ((ygk**2)*t/(2*M*N)) * ( 1 - (ygk**2/(12*N**2))*(5+3*t**2+eta2-9*eta2*t**2-4*eta2**2) + (ygk**4/(360*N**4))*(61+90*t**2+45*t**4) )
    lam_g = np.deg2rad(lam_zero) + (ygk/(N*np.cos(fi))) * ( 1 - (ygk**2/(6*N**2))*(1+2*t**2+eta2) + (ygk**4/(120*N**4))*(5+28*t**2+24*t**4+6*eta2+8*eta2*t**2) )
    return np.rad2deg(fi_g), np.rad2deg(lam_g)
    

In [592]:
def gk_to_geo2(xgk, ygk, lam_zero, a=6378137, e2=0.00669437999013, m0=1):
    """przliecza wsp w układzie gaussa krugera(metry) na geodezyjne(stopnie). lam_zero to południk osiowy(stopnie).
    Wzory z one nota."""
    a2 = a*a
    b2 = a2*(1 - e2)
    eprim2 = (a2 - b2) / b2
    A0 = 1 - (e2/4) - (3*(e2**2)/64) - (5*(e2**3)/256)
    A2 = (3/8) * (e2 + ((e2**2)/4) + (15*(e2**3))/128)
    A4 = (15/256) * (e2**2 + 3*(e2**3)/4)
    A6 = 35*(e2**3)/3072
    B_pop = xgk/(a*A0*m0)
    B = xgk/(a*A0*m0) + A2*np.sin(2*B_pop)/A0 - A4*np.sin(4*B_pop)/A0 + A6*np.sin(6*B_pop)/A0
    while abs(B-B_pop) > np.deg2rad(0.00001/3600):
        B_pop = B
        B = xgk/(a*A0*m0) + A2*np.sin(2*B_pop)/A0 - A4*np.sin(4*B_pop)/A0 + A6*np.sin(6*B_pop)/A0
    # B jest w radianach
    
    t = np.tan(B)
    eta2 = eprim2*np.cos(B)**2
    N = a / np.sqrt(1 - e2 * np.sin(B) ** 2)
    
    # B jest w radianach
    fi = B - (t/2) * ( ((ygk/(m0*N))**2)*(1+eta2) - (1/12)*((ygk/(m0*N))**4)*(5+3*(t**2)+6*eta2-6*eta2*(t**2)-3*(eta2**2)-9*(t**2)*(eta2**2)) + (1/360)*((ygk/(m0*N))**6)*(61+90*(t**2)+45*(t**4)+107*eta2-162*(t**2)*eta2-45*(t**4)*eta2) )
    l = (1/np.cos(B)) * ( ygk/(m0*N) - (1/6)*((ygk/(m0*N))**3)*(1+2*(t**2)+eta2) + (1/120)*((ygk/(m0*N))**5)*(5+28*(t**2)+24*(t**4)+6*eta2+8*eta2*(t**2)) )
    fi = np.rad2deg(fi)
    lam = lam_zero + np.rad2deg(l)
    return fi, lam


In [593]:
def get_mgk(fi, ygk, a=6378137, e2=0.00669437999013):
    """elementarna skala długości dla odwzorowania Gaussa-Krugera. one note
    fi, by policzyć N, M i R. ygk do wzoru."""
    fi = np.deg2rad(fi)
    N = a / np.sqrt(1 - e2 * np.sin(fi) ** 2)
    M = (a*(1-e2)) / ( (1 - e2 * np.sin(fi) ** 2) ** 1.5 )
    R = math.sqrt(M*N)
    mgk = 1 + ((ygk**2)/(2*(R**2))) + ((ygk**4)/(24*(R**4)))
    return mgk

In [594]:
def get_mgk2(fi, lam, lam_zero, a=6378137, e2=0.00669437999013):
    """elementarna skala długości dla odwzorowania Gaussa-Krugera. one note wersja 2.
    fi i lambdy w stopniach (wsp geo)"""
    a2 = a*a
    b2 = a2*(1 - e2)
    eprim2 = (a2 - b2) / b2 
    fi = np.deg2rad(fi)
    lam = np.deg2rad(lam)
    lam_zero = np.deg2rad(lam_zero)
    t = np.tan(fi)
    eta2 = eprim2 * (np.cos(fi))**2
    l = lam - lam_zero
    mgk = 1 + ((l**2)/2)*(np.cos(fi)**2)*(1+eta2) + ((l**4)/24)*(np.cos(fi)**4)*(5-4*t**2)
    return mgk

In [595]:
def from_1992_to_geo(x92, y92, a=6378137, e2=0.00669437999013):
    """przelicza x, y w układzie 1992 (x,y w metrach) na wsp geodezyjne."""
    # m0_92 - skala długości na południku osiowym 19 stopni
    m0_92 = 0.9993
#     x92 = m0_92*xgk - 5_300_000
#     y92 = m0_92*ygk + 500_000
    xgk = (x92 + 5_300_000)/m0_92
    ygk = (y92 - 500_000)/m0_92
    return gk_to_geo(xgk, ygk, 19, a, e2)
#     return gk_to_geo2(xgk, ygk, lam_zero=19)

In [596]:
def from_2000_to_geo(x20, y20, a=6378137, e2=0.00669437999013):
    """przelicza x,y  w układzie 2000 na wsp geodezyjne."""
    # m0_2000 - skala długości na południku osiowym
    m0_2000 = 0.999923
    nr_strefy = int(str(y20)[0])  # pierwsza cyfra wsp y to nr strefy 
    
#     x2000 = m0_2000 * xgk
#     y2000 = m0_2000 * ygk + nr_strefy*1_000_000 + 500_000
    
    xgk = x20/m0_2000
    ygk = (y20 - nr_strefy*1_000_000 - 500_000)/m0_2000
    return gk_to_geo(xgk, ygk, nr_strefy*3, a, e2)

In [597]:
def get_m92(x_92, y_92):
    """zwraca skalę długości w układzie wsp 2000, wejście to współrzędne x, y w układzie 1992."""
    m0_92 = 0.9993
    fi, lam = from_1992_to_geo(x_92, y_92)
    lam_zero = 19
    mgk = get_mgk2(fi, lam, lam_zero)
    return mgk*m0_92

In [598]:
def get_m2000(x_2000, y_2000):
    """zwraca skalę długości w układzie wsp 2000, wejście to współrzędne x, y w układzie 2000."""
    m0_2000 = 0.999923
    fi, lam = from_2000_to_geo(x_2000, y_2000)
    lam_zero = int(str(y_2000)[0])*3
    mgk = get_mgk2(fi, lam, lam_zero)
    return mgk*m0_2000

In [599]:
# zniekształcenia (w cm na 1km) w Excelu
# tabelka z wynikami też w Excelu

# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

## Testy

In [600]:
x = 5556698.103
y = 143059.986
f = gk_to_geo(x, y, 19)[0]
la = gk_to_geo(x, y, 19)[1]

In [601]:
get_mgk(f, y)

1.0002512595034179

In [602]:
get_mgk2(f, la, 19)

1.000251259400469

In [603]:
xgk = 5570120.597
ygk = 124812.228

x92 = 266221.513
y92 = 624724.859

x20 = 5568256.030
y20 = 7482170.562


In [604]:
to_gk(50.25, 20.75, 19)

(5570120.596980791, 124812.22773608562)

In [605]:
from_1992_to_geo(266221.513, 624724.859)

(50.250000003955606, 20.74999999778748)

In [606]:
from_2000_to_geo(5568256.030, 7482170.562)

(50.249999999939845, 20.74999999358122)

In [607]:
get_m92(266221.513, 624724.859)

0.9994911081961155

In [608]:
get_m2000(5568256.030, 7482170.562)

0.9999269026938475

In [609]:
to_gk(50, 21.25, 19)

(5543273.892000523, 161308.28340695688)

In [610]:
gk_to_geo(5543273.892000523, 161308.28340695688, 19)

(49.99999999995417, 21.25000000069796)

In [611]:
gk_to_geo2(5543273.892000523, 161308.28340695688, 19)

(49.999999999987956, 21.250000000697955)

In [612]:
np.tan(np.deg2rad(45))

0.9999999999999999

In [613]:
A = [50.25, 20.75]
B = [50, 20.75]
C = [50.25, 21.25]
D = [50, 21.25]
sr_szer = [50.125, 21]
srodkowy = [50.1252704344052, 21.0006510711438]

In [614]:
to_gk(D[0], D[1], 19)

(5543273.892000523, 161308.28340695688)

In [615]:
to_1992(D[0], D[1])

(239393.6002761228, 661195.367608572)

In [616]:
to_2000(D[0], D[1])

(5540450.350009964, 7517922.548450623)

In [617]:
pole_czworokata(A[0],A[1],D[0],D[1])

994265196.0743111

In [618]:
pole_poligonu([A,B,D,C])

0.125

# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

## Zestawienie współrzędnych

In [619]:
main_df = pd.read_csv('dane.csv', sep=';')

In [620]:
main_df['Xgk'] = main_df.apply(lambda row: to_gk(row[0], row[1], 19)[0], axis=1)
main_df['Ygk'] = main_df.apply(lambda row: to_gk(row[0], row[1], 19)[1], axis=1)
main_df['X1992'] = main_df.apply(lambda row: to_1992(row[0], row[1])[0], axis=1)
main_df['Y1992'] = main_df.apply(lambda row: to_1992(row[0], row[1])[1], axis=1)
main_df['X2000'] = main_df.apply(lambda row: to_2000(row[0], row[1])[0], axis=1)
main_df['Y2000'] = main_df.apply(lambda row: to_2000(row[0], row[1])[1], axis=1)

In [621]:
main_df

,fi,lam,Xgk,Ygk,X1992,Y1992,X2000,Y2000
0,50.25000,20.750000,5.570121e+06,124812.227736,266221.512563,624724.859177,5.568256e+06,7.482171e+06
1,50.00000,20.750000,5.542315e+06,125464.200846,238435.404992,625376.375906,5.540450e+06,7.482077e+06
2,50.25000,21.250000,5.571078e+06,160469.906665,267178.205642,660357.577730,5.568256e+06,7.517829e+06
3,50.00000,21.250000,5.543274e+06,161308.283407,239393.600276,661195.367609,5.540450e+06,7.517923e+06
4,50.12500,21.000000,5.556667e+06,143014.239309,252777.110759,642914.129342,5.554323e+06,7.500000e+06
5,50.12527,21.000651,5.556698e+06,143059.985842,252808.414212,642959.843852,5.554353e+06,7.500047e+06


In [622]:
ad1_df = main_df.drop(['fi', 'lam'], axis=1)
ad1_df.style.format('{:.3f}')

,Xgk,Ygk,X1992,Y1992,X2000,Y2000
0,5570120.597,124812.228,266221.513,624724.859,5568256.030,7482170.562
1,5542315.026,125464.201,238435.405,625376.376,5540450.350,7482077.452
2,5571077.960,160469.907,267178.206,660357.578,5568256.030,7517829.438
3,5543273.892,161308.283,239393.600,661195.368,5540450.350,7517922.548
4,5556666.778,143014.239,252777.111,642914.129,5554323.110,7500000.000
5,5556698.103,143059.986,252808.414,642959.844,5554353.189,7500046.554


## Zestawienie pól powierzchni

In [623]:
A = [50.25, 20.75]
B = [50, 20.75]
C = [50.25, 21.25]
D = [50, 21.25]
sr_szer = [50.125, 21]
srodkowy = [50.1252704344052, 21.0006510711438]
AGK = to_gk(A[0], A[1], 19)
BGK = to_gk(B[0], B[1], 19)
CGK = to_gk(C[0], C[1], 19)
DGK = to_gk(D[0], D[1], 19)
A92 = to_1992(A[0], A[1])
B92 = to_1992(B[0], B[1])
C92 = to_1992(C[0], C[1])
D92 = to_1992(D[0], D[1])
A20 = to_2000(A[0], A[1])
B20 = to_2000(B[0], B[1])
C20 = to_2000(C[0], C[1])
D20 = to_2000(D[0], D[1])

In [624]:
pola_df = pd.DataFrame()
pola_df['P_elip'] = [pole_czworokata(A[0],A[1],D[0],D[1])]
pola_df['P_GK'] = [pole_poligonu([AGK, BGK, DGK, CGK])]
pola_df['P_1992'] = [pole_poligonu([A92, B92, D92, C92])]
pola_df['P_2000'] = [pole_poligonu([A20, B20, D20, C20])]

In [625]:
pola_df  # pola w m^2

,P_elip,P_GK,P_1992,P_2000
0,9.942652e+08,9.947608e+08,9.933686e+08,9.941083e+08


In [626]:
ad2_df = pola_df.style.format('{:.6f}')
ad2_df

,P_elip,P_GK,P_1992,P_2000
0,994265196.074311,994760761.493312,993368583.859984,994108281.708822


## Zestawienie skal i zniekształceń długości

In [627]:
skale_df = pd.read_csv('skale.csv', sep=';')
skale_df

,Xgk,Ygk,X1992,Y1992,X2000,Y2000
0,5570120.597,124812.228,266221.513,624724.859,5568256.030,7482170.562
1,5542315.026,125464.201,238435.405,625376.376,5540450.350,7482077.452
2,5571077.960,160469.907,267178.206,660357.578,5568256.030,7517829.438
3,5543273.892,161308.283,239393.600,661195.368,5540450.350,7517922.548
4,5556666.778,143014.239,252777.111,642914.129,5554323.110,7500000.000
5,5556698.103,143059.986,252808.414,642959.844,5554353.189,7500046.554


In [628]:
# Z - zniekształcenie w [cm/km]. Ogólny wzór: Z = 1 - m_ukladu
skale_df['m_1992'] = skale_df.apply(lambda row: get_m92(row[2], row[3]), axis=1)
skale_df['Z_1992'] = skale_df.apply(lambda row: round((row[-1] - 1)*100000, 1), axis=1)
skale_df['m_2000'] = skale_df.apply(lambda row: get_m2000(row[4], row[5]), axis=1)
skale_df['Z_2000'] = skale_df.apply(lambda row: round((row[-1] - 1)*100000, 1), axis=1)
skale_df['m_gk'] = skale_df.apply(lambda row: get_mgk2(*gk_to_geo(row[0], row[1], int(str(row[1])[0])*3), int(str(row[1])[0])*3), axis=1)
skale_df['Z_gk'] = skale_df.apply(lambda row: round((row[-1] - 1)*100000, 1), axis=1)

In [629]:
skale_df.drop(columns=['Xgk', 'Ygk','X1992','Y1992','X2000','Y2000'], inplace=True)

In [630]:
skale_df

,m_1992,Z_1992,m_2000,Z_2000,m_gk,Z_gk
0,0.999491,-50.9,0.999927,-7.3,1.000191,19.1
1,0.999493,-50.7,0.999927,-7.3,1.000193,19.3
2,0.999616,-38.4,0.999927,-7.3,1.000316,31.6
3,0.999619,-38.1,0.999927,-7.3,1.000319,31.9
4,0.999551,-44.9,0.999923,-7.7,1.000251,25.1
5,0.999551,-44.9,0.999923,-7.7,1.000251,25.1


## Zestawienie skal i zniekształceń pól powierzchni

In [631]:
# Z^2 - zniekształcenie pola w [m^2/ha].
skale_df['m_1992^2'] = skale_df['m_1992']**2
skale_df['Z_1992^2'] = round((skale_df['m_1992^2'] - 1)*10000, 6)
skale_df['m_2000^2'] = skale_df['m_2000']**2
skale_df['Z_2000^2'] = round((skale_df['m_2000^2'] - 1)*10000, 6)
skale_df['m_gk^2'] = skale_df['m_gk']**2
skale_df['Z_gk^2'] = round((skale_df['m_gk^2'] - 1)*10000, 6)

In [632]:
skale_df.drop(columns=['m_1992','Z_1992','m_2000','Z_2000','m_gk','Z_gk'], inplace=True)

In [633]:
skale_df

,m_1992^2,Z_1992^2,m_2000^2,Z_2000^2,m_gk^2,Z_gk^2
0,0.998982,-10.175246,0.999854,-1.461893,1.000383,3.825207
1,0.998986,-10.135007,0.999854,-1.461071,1.000387,3.865503
2,0.999232,-7.680356,0.999854,-1.461893,1.000632,6.323594
3,0.999239,-7.613819,0.999854,-1.461071,1.000639,6.390225
4,0.999102,-8.979524,0.999846,-1.539941,1.000502,5.022605
5,0.999102,-8.976314,0.999846,-1.539940,1.000503,5.025819
